# My RNN Language Model implementation in Keras

In [1]:
import os

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GRU
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
print(tf.__version__)

2.0.0


In [2]:
with open('../datasets/username_dataset_256.txt', 'r') as file:
    data = file.readlines()
    
data = [example.strip().lower() for example in data]

with open('../datasets/username_dataset_256.txt', 'r') as file:
    data_str = file.read().lower()

vocab = list(set(data_str))
vocab.sort()

vocab.insert(0, '>') # End token: <end>
vocab.insert(0, '<') # Start token: <start>
vocab.insert(0, '*') # Padding

char_to_index = {ch: i for i, ch in enumerate(vocab)}
index_to_char = {i: ch for i, ch in enumerate(vocab)}

print(char_to_index)

longest_word = max(data, key=len)

print(f"Number of characters: {len(data_str)}")
print(f"Number of unique characters: {len(vocab)}")
print(f"Longest word: {longest_word}")
print(f"Length of longest word: {len(longest_word)}") # Needed information to know what length to pad the usernames to.

{'*': 0, '<': 1, '>': 2, '\n': 3, '1': 4, '2': 5, '3': 6, 'a': 7, 'b': 8, 'c': 9, 'd': 10, 'e': 11, 'f': 12, 'g': 13, 'h': 14, 'i': 15, 'j': 16, 'k': 17, 'l': 18, 'm': 19, 'n': 20, 'o': 21, 'p': 22, 'q': 23, 'r': 24, 's': 25, 't': 26, 'u': 27, 'v': 28, 'w': 29, 'x': 30, 'y': 31}
Number of characters: 1725
Number of unique characters: 32
Longest word: webmaster
Length of longest word: 9


In [3]:
def print_mapping(mapping):
    print('{')
    for char, _ in zip(mapping, range(20)):
        print('  {:4s}: {:3d},'.format(repr(char), mapping[char]))
    print('  ...\n}')
    
print_mapping(char_to_index)

{
  '*' :   0,
  '<' :   1,
  '>' :   2,
  '\n':   3,
  '1' :   4,
  '2' :   5,
  '3' :   6,
  'a' :   7,
  'b' :   8,
  'c' :   9,
  'd' :  10,
  'e' :  11,
  'f' :  12,
  'g' :  13,
  'h' :  14,
  'i' :  15,
  'j' :  16,
  'k' :  17,
  'l' :  18,
  'm' :  19,
  ...
}


In [4]:
def prepare_dataset(data, username_length):
    """Adds <start> and <end> tokens and pads the usernames."""
    data_X = []
    data_Y = []
    for username in data:
        pad = "*" * (username_length - len(username))
        X = np.array([char_to_index[char] for char in f"<{username}{pad}"]) #.reshape((username_length + 1, 1))
        Y = np.array([char_to_index[char] for char in f"{username}>{pad}"]) #.reshape((username_length + 1, 1))
        data_X.append(X)
        data_Y.append(Y)
    

    return (np.array(data_X), np.array(data_Y))

data = prepare_dataset(data, len(longest_word))

print(data[0].shape)
print(data[1].shape)
dataset = tf.data.Dataset.from_tensor_slices(data)

(256, 10)
(256, 10)


In [5]:
for input_example, target_example in dataset.take(1):
    print('Input data: ', repr(''.join([index_to_char[i] for i in input_example.numpy()])))
    print('Target data:', repr(''.join([index_to_char[i] for i in target_example.numpy()])))
    
for input_example, target_example in dataset.take(1):
    print('Input data: ', input_example.numpy())
    print('Target data:', target_example.numpy())

Input data:  '<info*****'
Target data: 'info>*****'
Input data:  [ 1 15 20 12 21  0  0  0  0  0]
Target data: [15 20 12 21  2  0  0  0  0  0]


In [6]:
BATCH_SIZE = 64
BUFFER_SIZE = 200
EPOCHS = 1000
VOCAB_SIZE = len(vocab)
RNN_UNITS = 512
EMBEDDING_DIM = 256

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset shapes: ((64, 10), (64, 10)), types: (tf.int32, tf.int32)>


In [7]:
def build_model(vocab_size, rnn_units, batch_size, embedding_dim, stateful=False):
    model = tf.keras.Sequential([
        tf.keras.layers.Masking(mask_value=0, batch_input_shape=[batch_size, None]),
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.GRU(rnn_units,
                             return_sequences=True,
                             stateful=stateful,
                             recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size, activation=tf.nn.softmax)])
    
    return model

In [8]:
model = build_model(
    vocab_size = VOCAB_SIZE,
    rnn_units=RNN_UNITS,
    embedding_dim=EMBEDDING_DIM,
    batch_size=BATCH_SIZE)

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (64, None)                0         
_________________________________________________________________
embedding (Embedding)        (64, None, 256)           8192      
_________________________________________________________________
gru (GRU)                    (64, None, 512)           1182720   
_________________________________________________________________
dense (Dense)                (64, None, 32)            16416     
Total params: 1,207,328
Trainable params: 1,207,328
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [11]:
model.fit(dataset, epochs=EPOCHS)

Epoch 1/1000
4/4 [==============================] - 4s 905ms/step - loss: 3.3477
Epoch 2/1000
4/4 [==============================] - 0s 86ms/step - loss: 2.8482
Epoch 3/1000
4/4 [==============================] - 0s 86ms/step - loss: 2.4851
Epoch 4/1000
4/4 [==============================] - 0s 89ms/step - loss: 2.2447
Epoch 5/1000
4/4 [==============================] - 0s 90ms/step - loss: 2.1735
Epoch 6/1000
4/4 [==============================] - 0s 93ms/step - loss: 2.1084
Epoch 7/1000
4/4 [==============================] - 0s 91ms/step - loss: 2.0349
Epoch 8/1000
4/4 [==============================] - 0s 92ms/step - loss: 2.0254
Epoch 9/1000
4/4 [==============================] - 0s 87ms/step - loss: 1.9826
Epoch 10/1000
4/4 [==============================] - 0s 86ms/step - loss: 1.9505
Epoch 11/1000
4/4 [==============================] - 0s 86ms/step - loss: 1.9158
Epoch 12/1000
4/4 [==============================] - 0s 86ms/step - loss: 1.8841
Epoch 13/1000
4/4 [=================

Epoch 102/1000
4/4 [==============================] - 0s 124ms/step - loss: 0.6161
Epoch 103/1000
4/4 [==============================] - 0s 103ms/step - loss: 0.6170
Epoch 104/1000
4/4 [==============================] - 0s 98ms/step - loss: 0.6112
Epoch 105/1000
4/4 [==============================] - 0s 101ms/step - loss: 0.6116
Epoch 106/1000
4/4 [==============================] - 0s 107ms/step - loss: 0.6088
Epoch 107/1000
4/4 [==============================] - 0s 102ms/step - loss: 0.6059
Epoch 108/1000
4/4 [==============================] - 0s 96ms/step - loss: 0.6053
Epoch 109/1000
4/4 [==============================] - 0s 96ms/step - loss: 0.6044
Epoch 110/1000
4/4 [==============================] - 0s 106ms/step - loss: 0.6050
Epoch 111/1000
4/4 [==============================] - 0s 98ms/step - loss: 0.6012
Epoch 112/1000
4/4 [==============================] - 0s 97ms/step - loss: 0.6002
Epoch 113/1000
4/4 [==============================] - 0s 88ms/step - loss: 0.5972
Epoch 114/

4/4 [==============================] - 0s 87ms/step - loss: 0.5674
Epoch 202/1000
4/4 [==============================] - 0s 83ms/step - loss: 0.5662
Epoch 203/1000
4/4 [==============================] - 0s 83ms/step - loss: 0.5693
Epoch 204/1000
4/4 [==============================] - 0s 84ms/step - loss: 0.5681
Epoch 205/1000
4/4 [==============================] - 0s 83ms/step - loss: 0.5648
Epoch 206/1000
4/4 [==============================] - 0s 83ms/step - loss: 0.5663
Epoch 207/1000
4/4 [==============================] - 0s 92ms/step - loss: 0.5660
Epoch 208/1000
4/4 [==============================] - 0s 86ms/step - loss: 0.5671
Epoch 209/1000
4/4 [==============================] - 0s 88ms/step - loss: 0.5677
Epoch 210/1000
4/4 [==============================] - 0s 88ms/step - loss: 0.5676
Epoch 211/1000
4/4 [==============================] - 0s 87ms/step - loss: 0.5652
Epoch 212/1000
4/4 [==============================] - 0s 88ms/step - loss: 0.5676
Epoch 213/1000
4/4 [===========

4/4 [==============================] - 0s 85ms/step - loss: 0.5627
Epoch 302/1000
4/4 [==============================] - 0s 92ms/step - loss: 0.5626
Epoch 303/1000
4/4 [==============================] - 0s 87ms/step - loss: 0.5634
Epoch 304/1000
4/4 [==============================] - 0s 90ms/step - loss: 0.5622
Epoch 305/1000
4/4 [==============================] - 0s 86ms/step - loss: 0.5621
Epoch 306/1000
4/4 [==============================] - 0s 91ms/step - loss: 0.5632
Epoch 307/1000
4/4 [==============================] - 0s 85ms/step - loss: 0.5628
Epoch 308/1000
4/4 [==============================] - 0s 92ms/step - loss: 0.5635
Epoch 309/1000
4/4 [==============================] - 0s 86ms/step - loss: 0.5623
Epoch 310/1000
4/4 [==============================] - 0s 93ms/step - loss: 0.5624
Epoch 311/1000
4/4 [==============================] - 0s 85ms/step - loss: 0.5631
Epoch 312/1000
4/4 [==============================] - 0s 88ms/step - loss: 0.5622
Epoch 313/1000
4/4 [===========

KeyboardInterrupt: 

In [12]:
# model.save_weights('../weights/gru_model_embedding_weights_200.h5')

In [13]:
model = build_model(VOCAB_SIZE, RNN_UNITS, 1, EMBEDDING_DIM, stateful=True)

In [14]:
model.load_weights('../weights/gru_model_embedding_weights_200.h5')

In [24]:
def generate_usernames(model, num_generate):
    generated_usernames = []

    for i in range(num_generate):
        model.reset_states()
        input_eval = np.array([char_to_index['<']]).reshape((1, 1))  # We start with the '<start>' token
        generated_username = []
        done = False
        
        while not done:
            predictions = model.predict(input_eval)
            predictions = tf.squeeze(predictions)

            predicted_id = np.random.choice(range(32), p=predictions.numpy())
            # predicted_id = np.argmax(predictions)

            input_eval = np.array([predicted_id]).reshape((1, 1))
            
            done = index_to_char[predicted_id] in ['>', '*']
            
            if not done:
                generated_username.append(index_to_char[predicted_id])
        generated_usernames.append("".join(generated_username))

    return generated_usernames

In [25]:
heatmap = dict()

In [26]:
for i in range(20000):
    generated_usernames = generate_usernames(model, 7)
    
    for username in generated_usernames:
        if heatmap.get(username) is not None:
            heatmap[username] += 1
        else:
            heatmap[username] = 1
    
    print(f'{generated_usernames}: {7 * i}')

['anthony', 'peanut', 'sex', 'rick', 'miller', 'allston', 'thunder']: 0
['peanut', 'paul', 'john', 'greg', 'shadow', 'natasha', 'null']: 7
['tarrant', 'horndog', 'dick', 'geler', 'james', 'andrew', 'craig']: 14
['tom', 'travis', 'freddy', 'gregory', 'jackie', 'billy', 'captain']: 21
['casper', 'adam', 'comtact', 'freddy', 'phantom', 'taylor', 'michael']: 28
['andy', 'diamond', 'blue', 'falcon', 'skippy', 'bob', 'marlboro']: 35
['richard', 'rabbit', 'falcon', 'phoenix', 'bitch', 'ryan', 'jackie']: 42
['golden', 'rick', 'dakota', 'russell', 'charles', 'mark', 'ronald']: 49
['jeremy', 'terry', 'maddog', 'hello', 'stephen', 'michael', 'maverick']: 56
['fuckyou', 'jeff', 'mike', 'michelle', 'tigger', 'philip', 'dakota']: 63
['horndog', 'cameron', 'maxwell', 'antonio', 'david', 'adrian', 'robert']: 70
['stevie', 'speedy', 'alan', 'phil', 'natasha', 'lesbian', 'matt']: 77
['baseball', 'aran', 'falcon', 'allston', 'roland', 'jimmy', 'coffee']: 84
['hardcore', 'free', 'onay', 'antonio', 'chuck'

['brandon', 'tommy', 'harry', 'billy', 'nathan', 'big', 'robert']: 777
['daniel', 'dallas', 'iceman', 'joshua', 'mickey', 'matthew', 'mark']: 784
['larry', 'orange', 'jackie', 'tiger', 'marcus', 'master', 'blonde']: 791
['golden', 'markus', 'coffee', 'knight', 'kenneth', 'superman', 'gary']: 798
['falcon', 'rick', 'randy', 'sexy', 'hunter', 'steven', 'anthony']: 805
['casper', 'phoenix', 'dennis', 'green', 'phoenix', 'jeffrey', 'miller']: 812
['robert', 'rick', 'peter', 'maddog', 'kevin', 'back', 'peanut']: 819
['anthony', 'robert', 'joe', 'kenneth', 'happy', 'richard', 'howard']: 826
['ranger', 'webmaster', 'letmein', 'pussy', 'webmaster', 'ronnie', 'paul']: 833
['billy', 'fuckme', 'amanda', 'arthur', 'horndog', 'danny', 'roland']: 840
['sparky', 'raymond', 'dave', 'arthur', 'johnny', 'gary', 'maxwell']: 847
['tigger', 'gary', 'sexy', 'general', 'robert', 'maddog', 'markus']: 854
['hardcore', 'sexy', 'stevie', 'roland', 'golden', 'taylor', 'fuck']: 861
['steve', 'bob', 'john', 'calvin

['mail', 'rocket', 'phil', 'matthew', 'gordon', 'markus', 'jake']: 1554
['bitch', 'nick', 'jeff', 'hard', 'bill', 'curious', 'miller']: 1561
['george', 'natasha', 'shadow', 'stephen', 'jimmy', 'black', 'casper']: 1568
['bob', 'james', 'taylor', 'marc', 'chicago', 'ranger', 'shadow']: 1575
['back', 'natasha', 'scotty', 'arthur', 'tommy', 'maverick', 'sex']: 1582
['raymond', 'back', 'paul', 'star', 'hello', 'markus', 'curious']: 1589
['mail', 'amanda', 'marlboro', 'avet', 'johnny', 'david', 'thunder']: 1596
['net', 'fuck', 'randy', 'james', 'player', 'alex', 'dave']: 1603
['donald', 'michael', 'big', 'chicken', 'maxwell', 'anthony', 'fucker']: 1610
['general', 'robert', 'joe', 'gordon', 'martin', 'star', 'james']: 1617
['root', 'silver', 'terry', 'thomas', 'jonathan', 'jackie', 'falcon']: 1624
['mail', 'sgar', 'summer', 'andy', 'scooter', 'brandon', 'jeffrey']: 1631
['darren', 'john', 'lucky', 'star', 'brian', 'bill', 'tory']: 1638
['robbie', 'kenneth', 'travis', 'free', 'parr', 'paul', 

['joseph', 'hunter', 'brandon', 'sunshine', 'golden', 'kevin', 'tommy']: 2324
['ryan', 'markus', 'maddog', 'matthew', 'john', 'harry', 'charles']: 2331
['roland', 'fucking', 'prince', 'bigboy', 'andrew', 'darren', 'big']: 2338
['info', 'scott', 'nick', 'raymond', 'trevie', 'marlboro', 'spider']: 2345
['patrick', 'lesbian', 'james', 'simon', 'michael', 'johnny', 'cameron']: 2352
['hotdog', 'maxwell', 'chris', 'letmein', 'marc', 'douglas', 'chuck']: 2359
['boomer', 'batman', 'johnson', 'silver', 'rocket', 'phoenix', 'captain']: 2366
['root', 'arthur', 'alan', 'dreamer', 'alex', 'joshua', 'batman']: 2373
['robert', 'monkey', 'letmein', 'andy', 'gary', 'fred', 'albert']: 2380
['mark', 'diamond', 'horndog', 'justin', 'teny', 'harley', 'amateur']: 2387
['oric', 'green', 'falcon', 'robert', 'horndog', 'blonde', 'jackson']: 2394
['danny', 'webmaster', 'fucking', 'robert', 'qwerty', 'bobby', 'contact']: 2401
['service', 'roland', 'gordon', 'doctor', 'boomer', 'joseph', 'michael']: 2408
['tigger

['baseball', 'bigdaddy', 'gary', 'austin', 'mike', 'stevie', 'terry']: 3080
['phoenix', 'billy', 'steve', 'steven', 'freddy', 'maxwell', 'joshua']: 3087
['superman', 'maxwell', 'xyon', 'jeff', 'james', 'tigger', 'natasha']: 3094
['james', 'diamond', 'chuck', 'big', 'harry', 'iceman', 'jack']: 3101
['tom', 'thomas', 'monkey', 'scooter', 'thomas', 'marlby', 'thomas']: 3108
['marc', 'pussy', 'hammer', 'michelle', 'dakota', 'martin', 'robert']: 3115
['maverick', 'qwerty', 'dave', 'darren', 'contact', 'cowboy', 'hard']: 3122
['brian', 'marlboro', 'peter', 'fucking', 'dick', 'david', 'summer']: 3129
['phil', 'martin', 'speedy', 'star', 'sales', 'mike', 'admin']: 3136
['jonathan', 'jeff', 'calvin', 'larry', 'rocket', 'robbie', 'nick']: 3143
['cowboy', 'cowboy', 'howard', 'amanda', 'football', 'brandon', 'dreamer']: 3150
['falcon', 'bigdaddy', 'adam', 'maverick', 'marc', 'jason', 'craig']: 3157
['robert', 'ranger', 'matthew', 'rabbit', 'fuckyou', 'bitch', 'orange']: 3164
['matt', 'frank', 'mon

...

['daniel', 'superman', 'taylor', 'peter', 'maverick', 'phue', 'jack']: 25858
['blonde', 'curtis', 'spider', 'darren', 'horndog', 'baseball', 'raymond']: 25865
['johnson', 'fred', 'nick', 'tony', 'brandon', 'blue', 'cunt']: 25872
['stevie', 'arthur', 'golden', 'batman', 'qwerty', 'tarrant', 'hammer']: 25879
['hotdog', 'hard', 'robert', 'summer', 'peter', 'daniel', 'tony']: 25886
['back', 'robert', 'charles', 'happy', 'purple', 'webmaster', 'gary']: 25893
['casper', 'blue', 'blue', 'cunt', 'terry', 'pass', 'taylor']: 25900
['randy', 'joe', 'knight', 'rodney', 'hardcore', 'maxwell', 'jackson']: 25907
['sunshine', 'simon', 'tommy', 'danny', 'bitch', 'chicken', 'james']: 25914
['cameron', 'fucking', 'kenneth', 'golden', 'phil', 'barney', 'mickey']: 25921
['mark', 'boomer', 'compaq', 'byando', 'player', 'samuel', 'darren']: 25928
['greg', 'nick', 'curious', 'jones', 'roger', 'travis', 'brandon']: 25935
['bubba', 'john', 'chad', 'cowboy', 'sjokey', 'andy', 'doladd']: 25942
['foctor', 'markus'

['adrian', 'tarrant', 'dallas', 'ranger', 'golden', 'contact', 'kenneth']: 26621
['charlie', 'andy', 'perer', 'coffee', 'jeffrey', 'adrian', 'love']: 26628
['sex', 'diamond', 'andy', 'simon', 'frank', 'skippy', 'maddog']: 26635
['chicken', 'mustang', 'star', 'bob', 'bigdick', 'calvin', 'dave']: 26642
['craig', 'charles', 'scooby', 'stuart', 'dick', 'hunter', 'amanda']: 26649
['cookie', 'compaq', 'spider', 'amateur', 'prince', 'dragon', 'johnny']: 26656
['rick', 'happy', 'james', 'summer', 'abc123', 'chris', 'curious']: 26663
['raymond', 'steven', 'rack', 'diamond', 'harrcor', 'service', 'ronnie']: 26670
['marlboro', 'frank', 'daniel', 'sexy', 'action', 'chris', 'porsche']: 26677
['travis', 'dick', 'david', 'sparky', 'shadow', 'dave', 'james']: 26684
['superman', 'jones', 'rick', 'dave', 'service', 'larry', 'robert']: 26691
['skippy', 'jeffrey', 'scooby', 'dave', 'fred', 'ranger', 'chris']: 26698
['', 'james', 'michelle', 'star', 'mark', 'justin', 'terry']: 26705
['rabbit', 'maddog', 'm

['dakota', 'marlboro', 'dennis', 'monkey', 'skipper', 'monkey', 'compaq']: 27370
['junior', 'jeremy', 'alex', 'brandon', 'roger', 'justin', 'timothy']: 27377
['craig', 'scooter', 'buster', 'bitch', 'jerry', 'taylor', 'alex']: 27384
['marlboro', 'chicago', 'johnny', 'thomas', 'eric', 'bob', 'skippy']: 27391
['nathan', 'general', 'smokey', 'dave', 'austin', 'general', 'jerry']: 27398
['chad', 'little', 'ryan', 'admin', 'hard', 'sexy', 'thomas']: 27405
['free', 'alan', 'curtis', 'sunshine', 'douglas', 'phil', 'kevin']: 27412
['casper', 'rabbit', 'golden', 'hotdog', 'jack', 'merlin', 'super']: 27419
['blue', 'maddog', 'simon', '', 'bigdaddy', 'dragon', 'hunter']: 27426
['mark', 'marc', 'badboy', 'iceman', 'thomas', 'albert', 'bobby']: 27433
['roger', 'samuel', 'back', 'monster', 'ryan', 'joshua', 'scotty']: 27440
['pussy', 'larry', 'happy', 'stevie', 'cunt', 'boomer', 'harry']: 27447
['adrian', 'marc', 'thomas', 'green', 'admin', 'admin', 'douglas']: 27454
['billy', 'scotty', 'runner', 'ga

['hardcore', 'jackie', 'michael', 'buster', 'ronnie', 'smokey', 'sdavis']: 28126
['stephen', 'sexy', 'phil', 'gary', 'monkey', 'badboy', 'chuck']: 28133
['richard', 'natasha', 'johnny', 'speedy', 'little', 'bobby', 'dick']: 28140
['jonathan', 'brian', 'marc', 'mustang', 'buster', 'dallas', 'albert']: 28147


KeyboardInterrupt: 

In [27]:
print(len(heatmap))
for username, occurence in sorted(heatmap.items(),
                                  key=lambda item: item[1],
                                  reverse=True):
    print(f'{username}: {occurence}')

819
robert: 290
michael: 256
thomas: 172
johnny: 171
billy: 163
maxwell: 163
jackie: 158
bigboy: 158
steven: 155
johnson: 155
charles: 153
phil: 150
anthony: 148
raymond: 148
dakota: 144
simon: 144
tigger: 143
roland: 141
darren: 141
marc: 140
star: 140
greg: 138
back: 138
dallas: 138
chicago: 137
arthur: 137
brandon: 136
joe: 136
alex: 135
monkey: 134
bigdick: 133
dragon: 133
stuart: 131
bobby: 130
marlboro: 129
maverick: 128
timothy: 128
joshua: 128
david: 127
harley: 127
harry: 127
sexy: 127
peanut: 126
allston: 126
samuel: 126
tommy: 126
craig: 125
falcon: 125
richard: 125
pass: 125
jones: 125
robbie: 125
nick: 124
killer: 123
bigdaddy: 123
summer: 123
natasha: 122
black: 122
porsche: 121
barney: 121
kenneth: 121
eric: 121
love: 121
freddy: 120
battery: 120
morgan: 120
fred: 120
chicken: 119
mark: 118
mickey: 118
gary: 118
superman: 118
hard: 118
ronnie: 118
blue: 117
douglas: 117
charlie: 117
admin: 117
big: 117
shadow: 116
bubba: 116
maddog: 114
smokey: 114
purple: 114
silver: 11